## Okay, just one more neat Julia language feature: generating code with code

(depending on how fast I run through these slides)

At some point I developed an interest in sieving prime numbers, and got particularly interested in what is probably the fastest implementation of the Sieve of Erathostenes:

https://github.com/kimwalisch/primesieve/

It implements this:

![Prime sieve](https://upload.wikimedia.org/wikipedia/commons/b/b9/Sieve_of_Eratosthenes_animation.gif)

but skips multiples of 2, 3, and 5, does something cache efficient, and has some ingenious code to unroll the "inner sieving loop". It's great but it's quite something to look at:

https://github.com/kimwalisch/primesieve/blob/master/src/EratSmall.cpp#L93-L314

Writing this code is extremely tedious, but writing the code that generates the code is much easier!

So I tried that: https://github.com/haampie/FastPrimeSieve.jl/blob/master/src/generate_sieving_loop.jl#L108 -- although still very difficult to follow, and it could potentially be used to generalize the loops

In [1]:
] activate .

  Activating environment at `~/tutorial_julia_interactive/01_more_cpu_before_the_gpu/Project.toml`


In [2]:
] dev https://github.com/haampie/FastPrimeSieve.jl

     Cloning git-repo `https://github.com/haampie/FastPrimeSieve.jl`
Path `/scratch/snx3000/hstoppel/julia_dev/FastPrimeSieve` exists and looks like the correct repo. Using existing path.
   Resolving package versions...
  No Changes to `~/tutorial_julia_interactive/01_more_cpu_before_the_gpu/Project.toml`
  No Changes to `~/tutorial_julia_interactive/01_more_cpu_before_the_gpu/Manifest.toml`


In [3]:
function how_many_prime_numbers_less_than_or_equal_to(n)
    is_prime = trues(n)
    is_prime[1] = false
    @inbounds for i=2:isqrt(n)
        if is_prime[i]
            for j=i^2:i:n
                is_prime[j] = false
            end
        end
    end
    return count(is_prime)
end

how_many_prime_numbers_less_than_or_equal_to (generic function with 1 method)

In [4]:
@time how_many_prime_numbers_less_than_or_equal_to(10^10)

 90.219791 seconds (3 allocations: 1.164 GiB, 0.01% gc time)


455052511

In [5]:
using FastPrimeSieve: countprimes, pcountprimes

In [14]:
@time countprimes(10^10, segment_length=32*1024)

  2.944059 seconds (7 allocations: 277.109 KiB)


455052511

In [7]:
# Build with Cray's GNU GCC programming environment
cd(ENV["SCRATCH"]) do
    rm("primesieve", force=true, recursive=true)
    run(`git clone https://github.com/kimwalisch/primesieve.git`)
    cd("primesieve") do
        # latest release
        run(`git checkout da47d8259f2dbab7e0852eb8ed849bde45bdbe65`)
        mkdir("build")
        cd("build") do
            run(`cmake .. -DCMAKE_BUILD_TYPE=Release -DCMAKE_CXX_COMPILER=CC -DCMAKE_CXX_FLAGS="-march=native"`)
            run(`make -j`)
        end
    end
end

Cloning into 'primesieve'...
Note: switching to 'da47d8259f2dbab7e0852eb8ed849bde45bdbe65'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at da47d825 Use future-proof CMake


-- The CXX compiler identification is GNU 9.3.0
-- Cray Programming Environment 2.7.3 CXX
-- Check for working CXX compiler: /opt/cray/pe/craype/2.7.3/bin/CC
-- Check for working CXX compiler: /opt/cray/pe/craype/2.7.3/bin/CC -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test atomic64
-- Performing Test atomic64 - Success
-- Configuring done
-- Generating done
-- Build files have been written to: /scratch/snx3000/hstoppel/primesieve/build
Scanning dependencies of target libprimesieve
[ 17%] Building CXX object CMakeFiles/libprimesieve.dir/src/EratSmall.cpp.o
[ 19%] Building CXX object CMakeFiles/libprimesieve.dir/src/EratMedium.cpp.o
[ 21%] Building CXX object CMakeFiles/libprimesieve.dir/src/EratBig.cpp.o
[ 23%] Building CXX object CMakeFiles/libprimesieve.dir/src/iterator-c.cpp.o
[ 23%] Building CXX object CMakeFiles/libprimesieve.dir/src/Erat.cpp.o
[ 23%] 

Process(`make -j`, ProcessExited(0))

In [12]:
primesieve = joinpath(ENV["SCRATCH"], "primesieve", "build", "primesieve")
cmd = `$(primesieve) 1e10 -t 12 -s 32`

run(cmd);

Sieve size = 32 KiB
Threads = 12
100%
Seconds: 0.199
Primes: 455052511


In [13]:
@time pcountprimes(10^10, segment_length=32*1024)

  0.304542 seconds (185 allocations: 2.518 MiB)


455052511

Clearly `primesieve` is still quite a bit faster (0.2s vs 0.3s) and it will easily win in the $2^{32}$ to $2^{64}$ where you have to switch strategies! But then again, I definitely did not spent the same amount of time on writing it.

If you go down the rabbithole and run

```julia
using FastPrimeSieve: SmallSieve

@code_native debuginfo=:none SmallSieve(10^10)
```

you will in fact see the same "Highly optimized inner loop" as advertized in https://github.com/kimwalisch/primesieve/blob/master/doc/ALGORITHMS.md#highly-optimized-inner-loop:

```asm
L6688:
    andb    $-65, -1(%rcx,%rax)
    andb    $-5, (%r8,%rax)
    andb    $-9, (%r9,%rax)
    andb    $127, (%r11,%rax)
    andb    $-2, (%rbp,%rax)
    andb    $-17, (%rbx,%rax)
    andb    $-33, (%rsi,%rax)
    andb    $-3, (%rdx,%rax)
    addq    %r10, %rax
    cmpq    %rax, %rdi
    jge    L6688

```